# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

This data represents 17 campaigns with 79354 contacts gathered. The campaigns were all phone campaigns conducted between May 2008 and November 2010.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [255]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC 
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.metrics import recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [256]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [257]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The business problem is to correctly identify the customers who are likely to subscribe to the term deposit. The bank can then target these customers with promotional offers. The bank can also use the model to identify the key drivers that influence the customers' decision to subscribe to the term deposit.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [259]:
bank_df = df[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'y']].copy()
bank_df['y'].value_counts()


no     36548
yes     4640
Name: y, dtype: int64

In [260]:
#Clean and one hot encode the data
bank_df = pd.get_dummies(bank_df, columns=['job', 'marital', 'education', 'contact'], drop_first=True)
bank_df['y'] = bank_df['y'].map({'yes': 1, 'no': 0})
bank_df['housing'] = bank_df['housing'].map({'yes': 1, 'no': 0})
bank_df['default'] = bank_df['default'].map({'yes': 1, 'no': 0})
bank_df['loan'] = bank_df['loan'].map({'yes': 1, 'no': 0})
bank_df.dropna(inplace=True)
bank_df.head()

,default,housing,loan,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,contact_telephone
0,0.0,0.0,0.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,1.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0.0,0.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
6,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [261]:
X = bank_df.drop('y', axis=1)
y = bank_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

The stated success rate of the marketing campaign was 8%, with 79354 contacts and 6499 successes. We should aim to be able to identify these successful candidates as best as possible. To determine a baseline, we will use the sklearn DummyClassifier(). We will use uniform predictions

In [262]:
baseline = DummyClassifier(strategy='stratified').fit(X_train, y_train)
baseline_predictions = baseline.predict(X_test)
baseline_accuracy = accuracy_score(baseline_predictions, y_test)
print(f'Baseline accuracy score: {baseline_accuracy}')


Baseline accuracy score: 0.7772541627395538


We will attempt to beat the accuracy score of: 0.7706566132579328

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [263]:
logistic = LogisticRegression().fit(X_train, y_train)
logistic_predictions = logistic.predict(X_test)
logisitic_train_predictions = logistic.predict(X_train)


### Problem 9: Score the Model

What is the accuracy of your model?

In [264]:
logistic_accuracy = accuracy_score(logistic_predictions, y_test)
print(f'Logistic accuracy score: {logistic_accuracy}')

Logistic accuracy score: 0.8704052780395853


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [265]:
#Logistic Regression
t0 = time.time()

logistic = LogisticRegression().fit(X_train, y_train)

logistic_predictions = logistic.predict(X_test)
logisitic_train_predictions = logistic.predict(X_train)

t1 = time.time()

logistic_accuracy = accuracy_score(logistic_predictions, y_test)
logistic_train_accuracy = accuracy_score(logisitic_train_predictions, y_train)

print(f'Logistic accuracy score: {logistic_accuracy}')
print(f'Logistic train accuracy score: {logistic_train_accuracy}')
print(f'Logistic time to train: {t1-t0}')

Logistic accuracy score: 0.8704052780395853
Logistic train accuracy score: 0.8712984054669703
Logistic time to train: 0.15399909019470215


In [266]:
#KNN
t2 = time.time()

knn = KNC().fit(X_train.values, y_train.values)

knn_train_predictions = knn.predict(X_train.values)
knn_predictions = knn.predict(X_test.values)

t3 = time.time()

knn_train_accuracy = accuracy_score(knn_train_predictions, y_train)
knn_accuracy = accuracy_score(knn_predictions, y_test)

print(f'KNN accuracy score: {knn_accuracy}')
print(f'KNN train accuracy score: {knn_train_accuracy}')
print(f'KNN time to train: {t3-t2}')

KNN accuracy score: 0.8517122211749921
KNN train accuracy score: 0.8596732385515671
KNN time to train: 1.4708962440490723


In [267]:
#SVM
t4 = time.time()

svm = SVC().fit(X_train, y_train)

svm_train_predictions = svm.predict(X_train)
svm_predictions = svm.predict(X_test)
t5 = time.time()

svm_train_accuracy = accuracy_score(svm_train_predictions, y_train)
svm_accuracy = accuracy_score(svm_predictions, y_test)

print(f'SVM accuracy score: {svm_accuracy}')
print(f'SVM train accuracy score: {svm_train_accuracy}')
print(f'SVM time to train: {t5-t4}')


SVM accuracy score: 0.8704052780395853
SVM train accuracy score: 0.8713376796795225
SVM time to train: 37.109230041503906


In [268]:
#DTC
t6 = time.time()

dtc = DTC().fit(X_train, y_train)

dtc_train_predictions = dtc.predict(X_train)
dtc_predictions = dtc.predict(X_test)
t7 = time.time()

dtc_train_accuracy = accuracy_score(dtc_train_predictions, y_train)
dtc_accuracy = accuracy_score(dtc_predictions, y_test)

print(f'DTC accuracy score: {dtc_accuracy}')
print(f'DTC train accuracy score: {dtc_train_accuracy}')
print(f'DTC time to train: {t7-t6}')



DTC accuracy score: 0.8660069117185045
DTC train accuracy score: 0.8752651009347262
DTC time to train: 0.04367685317993164


In [269]:
results = pd.DataFrame({
  'Model': ['Logistic', 'KNN', 'SVM', 'DTC'],
  'Test Accuracy': [logistic_accuracy, knn_accuracy, svm_accuracy, dtc_accuracy],
  'Train Accuracy': [logistic_train_accuracy, knn_train_accuracy, svm_train_accuracy, dtc_train_accuracy],
  'Time to Train': [t1-t0, t3-t2, t5-t4, t7-t6]
  })

print(results)

      Model  Test Accuracy  Train Accuracy  Time to Train
0  Logistic       0.870405        0.871298       0.153999
1       KNN       0.851712        0.859673       1.470896
2       SVM       0.870405        0.871338      37.109230
3       DTC       0.866007        0.875265       0.043677


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

### Does scaling have an effect on the models?


In [270]:
#Store models in list for testing
log = LogisticRegression()
svm = SVC()
knn = KNC()
dtc = DTC()

testing = [log, svm, knn, dtc]

#Test models
for model in testing:
  pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
  ])
  pipeline.fit(X_train, y_train)

  predictions = pipeline.predict(X_test)
  accuracy = accuracy_score(predictions, y_test)
  print(f'{model} accuracy: {accuracy}')
  

LogisticRegression() accuracy: 0.8704052780395853
SVC() accuracy: 0.8704052780395853
KNeighborsClassifier() accuracy: 0.8502984605717876
DecisionTreeClassifier() accuracy: 0.8660069117185045


#### Scaling does not have an effect


In [272]:
### Does dropping the gender column improve the model?

##### Questions